Leaderboard
===========

Basic analysis and stats for private leaderboards.

Private leaderboards are available at the url in the form:  
https://adventofcode.com/2021/leaderboard/private/view/{leaderboard_id}.json

### Instructions  
1) Visit your private leaderboard at the json link  
2) Save the json to the json folder  
3) Update the path_leaderboard to point to the leaderboard json file

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [ ]:
path_leaderboard = 'json/1491207_20211206.json'
with open (path_leaderboard, 'r') as fh:
    data = json.load(fh)


In [ ]:
# Members
members = []
for member_id in data['members']:
    member_name = data['members'][member_id].get('name')
    member_local_score = data['members'][member_id].get('local_score')
    member = {
        "name": member_name,
        "id": member_id,
        "local_score": member_local_score
    }
    members.append(member)

df_members = pd.DataFrame.from_dict(members)
df_members.sort_values(['local_score'], ascending=False)

member_count = df_members['id'].nunique()

member_count

In [ ]:
# Scores
scores = []

for member in data['members']:

    
    for day in data['members'][member].get('completion_day_level'):

        
        for star in data['members'][member].get('completion_day_level',{0: {}}).get(day, {}):
            scores.append({
                'id': member, 
                'name': data['members'][member].get('name'),
                'day': day,
                'star': star,
                'ts': data['members'][member].get('completion_day_level').get(day).get(star)['get_star_ts']
            })

df_scores = pd.DataFrame.from_dict(scores)
df_scores

In [ ]:
df_scores['time'] = df_scores['ts'].apply(lambda x: pd.to_datetime(x, unit='s'))
df_scores[df_scores['day'] == '5'].sort_values(['time'])

In [ ]:
# Calculate points
df_scores["part_rank"] = df_scores.groupby(['day', 'star'])["ts"].rank("dense", ascending=True).astype('int')
df_scores["points"] = df_scores['part_rank'].apply(lambda x: (member_count + 1) - x).astype('int')
df_scores[df_scores['day'] == '6'].sort_values(['time'], ascending=True)

In [ ]:
# Cumulative points over time
df_scores['c_points'] = df_scores.sort_values(['ts']).groupby(['id'])['points'].cumsum()

In [ ]:
df_scores.sort_values(['c_points'], ascending=False).head(10)

In [ ]:
# Leaderboard
df_scores.groupby(df_scores['name'])['c_points'].max().sort_values(ascending=False)

In [ ]:
# When does a member submit Part 2 by?

def time_in_seconds(dt):
    midnight = dt.replace(hour=5, minute=0, second=0, microsecond=0)
    seconds = (dt - midnight).seconds    
    return seconds

df_scores['time_seconds'] = df_scores['time'].apply(lambda x: time_in_seconds(x))
df_scores[df_scores['star'] == '2'].groupby(['name'])['time_seconds'].min().apply(lambda x: pd.Timedelta(seconds=x + 5 * 60 * 60)).sort_values()

In [ ]:
df_scores[df_scores['star'] == '2']